# 1 - Data preparations and Random Forest model

# Backround

The article uses the highly used and preprocessed bioactivity dataset from the ChEMBL database, version 20. More specifically these seven different bioactivity classes:

(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In this project we will mainly focus on (a) the ChEMBL205 and (f) ChEMBL218 as these got quite different results from each other. Also 

In [20]:
#hide
!nvidia-smi

Mon Apr 26 14:10:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   32C    P8     6W / 180W |      1MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#hide
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
%%capture
#hide
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

In [3]:
#hide
from rdkit import Chem
from rdkit.Chem import AllChem

In [6]:
#hide
path = Path('dataset/13321_2017_226_MOESM1_ESM/')
#df = pd.read_csv('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv', index_col=0)

In [7]:
#hide
#df.head()
list(path.iterdir())

[PosixPath('dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl_ecfp_1024.csv'),
 PosixPath('dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv')]

First we need to convert the smiles from the dataset to ECFP fingerprint which is hashed into 1024 length bits. For this we made a function for creating a fingerprint using the methods from the RDKit library as follows: 

In [8]:
def fp(smile, diam = 2, bits = 1024):

    mol = Chem.MolFromSmiles(smile)
    Chem.SanitizeMol(mol)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, diam, nBits = bits) 
    return fp

We then define a method for converting the csv with smiles to also have the fingerprint stored as colums from ECFP_1 to ECFP_1024 for each row. 

In [10]:
#ECFP4
#Generated Circular fingerprints hashed into n bits length vectors.

def ECFP(ifile, ofile, diam, bits):
    
    print(f"Making fingerprints for file: {ifile}")
    df = pd.read_csv(ifile)
    
    df.insert(2, "ECFP4_", df.SMILES.apply(fp))
    
    df[[f"ECFP4_{i+1}" for i in range(len(df.ECFP4_[0]))]] = df.ECFP4_.to_list()
    
    df.drop("ECFP4_", axis = 1, inplace = True)
    
    
    df.to_csv(path/ofile, index = None)
    return df

Here we can specify the dataset we want to use and to run the ECFP function on. 

In [11]:
dataset='CHEMBL205_cl'

In [12]:
#hide_output
#ECFP(path/f'{dataset}.csv', f'./{dataset}_ecfp_1024.csv', 2, 1024)

Next we create a dataframe from the newly generated csv file. 

In [13]:
df = pd.read_csv(path/f'{dataset}_ecfp_1024.csv')

In [14]:
df.head()

,CID,SMILES,Activity,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,...,ECFP4_1015,ECFP4_1016,ECFP4_1017,ECFP4_1018,ECFP4_1019,ECFP4_1020,ECFP4_1021,ECFP4_1022,ECFP4_1023,ECFP4_1024
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 1027 entries, CID to ECFP4_1024
dtypes: int64(1025), object(2)
memory usage: 140.6+ MB


In [16]:
X, y = df.drop(["CID", "SMILES", "Activity"], axis=1), df["Activity"]

In [17]:
#hide
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 140.2 MB


In [18]:
#hide
y.head(), y.size, type(y)

(0    1
 1    1
 2    1
 3    1
 4    1
 Name: Activity, dtype: int64,
 17941,
 pandas.core.series.Series)

# Train test split

In [21]:
#hide
from sklearn.model_selection import train_test_split, KFold

In [22]:
# regular train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

# 5-Fold Cross Validation

Follows the article where the data is split using 5-fold cross validation. 

In [23]:
#5-fold
kf = KFold(n_splits=5, shuffle=True, random_state=999)

In [24]:
X_train_list, X_valid_list, y_train_list, y_valid_list = [], [], [], []

for train_index, valid_index in kf.split(X_train):
    X_train_list.append(X_train.iloc[train_index])
    X_valid_list.append(X_train.iloc[valid_index])
    y_train_list.append(y_train.iloc[train_index])
    y_valid_list.append(y_train.iloc[valid_index])

In [25]:
#hide
y_train_list[0].head()

14960    0
16280    0
16068    0
13692    0
14043    0
Name: Activity, dtype: int64

In [26]:
#hide
X_train_list[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8611 entries, 14960 to 10114
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 67.3 MB


# Random Forest comparison

In [27]:
#hide
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid

In [28]:
#hide
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

In [29]:
#hide
top_mcc_scores = {
    
    'CHEMBL205': 0.862,
    'CHEMBL301': 0.926,
    'CHEMBL240': 0.884,
    'CHEMBL219': 0.887,
    'CHEMBL244': 0.983,
    'CHEMBL218': 0.941,
    'CHEMBL1978': 0.904
}

Here, a Random Forest model is used, the settings from the paper is used with going through the different setting in each CV iteration. A Random Forest model is usually a very good performance model for classification, that is the main reason for its usage here. 

In [30]:
param_grid = {
    'n_estimators': [10,50,100,200,300,700], 
    'criterion': ['gini', 'entropy'],
    'max_features': ['log2', 'sqrt']
             }

param_grid = ParameterGrid(param_grid)

Function for training a Random Forest classifier with the specified parameters, default: n_estimators=5, criterion='gini', max_features='log2'

In [31]:
# train method for Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators=5, criterion='gini', max_features='log2'):

    
    rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, min_samples_split=2, max_features=max_features, 
                               max_leaf_nodes=None,bootstrap=False,oob_score=False, n_jobs=-1, random_state=69)
    
    rf.fit(X_train,y_train)
    y_pred= rf.predict(X_test)
    y_pred_prob=rf.predict_proba(X_test)
    
    temp=[]
    for j in range(len(y_pred_prob)):
        temp.append(y_pred_prob[j][1])
    auc=roc_auc_score(np.array(y_test),np.array(temp))
    acc2=accuracy_score(y_test,y_pred)
    mcc=matthews_corrcoef(y_test,y_pred)
    Recall=recall_score(y_test, y_pred,pos_label=1)
    Precision=precision_score(y_test, y_pred,pos_label=1)
    F1_score=f1_score(y_test, y_pred,pos_label=1)

    return auc,acc2,mcc,Recall,Precision,F1_score

Next we test all the different settings on the random forest model

In [35]:
for setting in param_grid:
    print(f"Testing combination: {setting}")
    
    
    aucs, accs, mccs, recalls, precs, f1_scores = [], [], [], [], [], []
   
    for i in range(0,5):
    
        X_train = X_train_list[i]
        X_valid = X_valid_list[i]
        y_train = y_train_list[i]
        y_valid = y_valid_list[i]
    
        auc,acc2,mcc,recall,precision,F1_score = train_rf(X_train, X_test, y_train, y_test, 
                                                             n_estimators=setting['n_estimators'], 
                                                             criterion=setting['criterion'],
                                                             max_features=setting['max_features'])
        
        aucs.append(auc)
        accs.append(acc2)
        mccs.append(mcc)
        recalls.append(recall)
        precs.append(precision)
        f1_scores.append(F1_score)
        
    print(f"Average ROCAUC of the folds: {np.mean(aucs)}")
    print(f"Average accuracy of the folds: {np.mean(accs)}")
    print(f"Average Matthews correlation of the folds: {np.mean(mccs)}")
    print(f"Average recall of the folds: {np.mean(recalls)}")
    print(f"Average precision of the folds: {np.mean(precs)}")
    print(f"Average f1 score of the folds: {np.mean(f1_scores)}")
    print()

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 10}
Average ROCAUC of the folds: 0.9772882110994567
Average accuracy of the folds: 0.9667270447262087
Average Matthews correlation of the folds: 0.7860500038292063
Average recall of the folds: 0.7372307692307694
Average precision of the folds: 0.8760069960166821
Average f1 score of the folds: 0.800503358063245

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 50}
Average ROCAUC of the folds: 0.9883235554088932
Average accuracy of the folds: 0.9716316009474711
Average Matthews correlation of the folds: 0.8209589646616985
Average recall of the folds: 0.7926153846153847
Average precision of the folds: 0.8822409748917097
Average f1 score of the folds: 0.83500841879966

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}
Average ROCAUC of the folds: 0.9897701382423307
Average accuracy of the folds: 0.971743068134318
Average Matthews correla

Average ROCAUC of the folds: 0.9899568655643423
Average accuracy of the folds: 0.9726348056290929
Average Matthews correlation of the folds: 0.8306298858661861
Average recall of the folds: 0.8258461538461539
Average precision of the folds: 0.865803297784208
Average f1 score of the folds: 0.8453519472560641

Testing combination: {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 300}
Average ROCAUC of the folds: 0.9901267869559582
Average accuracy of the folds: 0.9724118712553992
Average Matthews correlation of the folds: 0.8293221619482496
Average recall of the folds: 0.8252307692307692
Average precision of the folds: 0.8640346330095527
Average f1 score of the folds: 0.8441788146153677

Testing combination: {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 700}
Average ROCAUC of the folds: 0.9903818693945858
Average accuracy of the folds: 0.9726626724258047
Average Matthews correlation of the folds: 0.8309066881967601
Average recall of the folds: 0.82676923

Next, we choose the settings that gave us the best score and train a the model on these on all the folds. The results for each cross validation is displayed and the MCC compared to the best average score from the paper. 

In [36]:
MCC=[]

In [38]:
#collapse_output

# Train and print scores for each split
for i in range(0,5):
    
    print(f'Cross validation iteration: {i + 1}')
    
    X_train = X_train_list[i]
    X_valid = X_valid_list[i]
    y_train = y_train_list[i]
    y_valid = y_valid_list[i]
    
    auc,acc2,mcc,Recall,Precision,F1_score = train_rf(X_train, X_valid, y_train, y_valid, 
                                                      n_estimators=200, criterion='entropy', max_features='sqrt')
    MCC.append(mcc)
    
    print(f'AUC score:{auc}')
    print(f'Accuracy: {acc2}')
    print(f'Matthews: {mcc}')
    print(f'Recall: {Recall}')
    print(f'Precision: {Precision}')
    print(f'F1 score: {F1_score}')
    print()

Cross validation iteration: 1
AUC score:0.9895547345611034
Accuracy: 0.9716674407803065
Matthews: 0.8322885854488427
Recall: 0.8415841584158416
Precision: 0.8542713567839196
F1 score: 0.8478802992518704

Cross validation iteration: 2
AUC score:0.9881408694710627
Accuracy: 0.9772410589874594
Matthews: 0.8543605077494866
Recall: 0.8457446808510638
Precision: 0.888268156424581
F1 score: 0.8664850136239782

Cross validation iteration: 3
AUC score:0.99115142437103
Accuracy: 0.972131908964236
Matthews: 0.8182680063069008
Recall: 0.819672131147541
Precision: 0.847457627118644
F1 score: 0.8333333333333333

Cross validation iteration: 4
AUC score:0.9761998796790828
Accuracy: 0.9684161634928008
Matthews: 0.8252479636345342
Recall: 0.7964601769911505
Precision: 0.8910891089108911
F1 score: 0.8411214953271028

Cross validation iteration: 5
AUC score:0.992038545211134
Accuracy: 0.9763011152416357
Matthews: 0.8437545323349981
Recall: 0.8351648351648352
Precision: 0.8786127167630058
F1 score: 0.85633

In [39]:
np.mean(MCC), top_mcc_scores['CHEMBL205']

(0.8360720667107868, 0.862)